In [4]:
import s3fs
import xarray as xr
import os
import requests
import sys

In [6]:
! cat ~/.netrc

machine urs.earthdata.nasa.gov login cssprad1 password Spra9209!


In [7]:
def get_temp_creds(provider):
    
    s3_cred_endpoint = {
    'podaac':'https://archive.podaac.earthdata.nasa.gov/s3credentials',
    'gesdisc': 'https://data.gesdisc.earthdata.nasa.gov/s3credentials',
    'lpdaac':'https://data.lpdaac.earthdatacloud.nasa.gov/s3credentials',
    'ornldaac': 'https://data.ornldaac.earthdata.nasa.gov/s3credentials',
    'ghrcdaac': 'https://data.ghrc.earthdata.nasa.gov/s3credentials'
    }
    
    return requests.get(s3_cred_endpoint[provider]).json()

In [12]:
temp_creds_req = get_temp_creds('gesdisc')
fs = s3fs.S3FileSystem(anon=False, 
                          key=temp_creds_req['accessKeyId'], 
                          secret=temp_creds_req['secretAccessKey'], 
                          token=temp_creds_req['sessionToken'])

## Example of getting a list of SSH urls; Can be replaced by CMR 
short_name = 'GLDAS/GLDAS_NOAH025_3H.2.1/2019/138'
bucket = os.path.join('s3://gesdisc-cumulus-prod-protected/', short_name, 'GLDAS_NOAH025_3H.A20190518*.nc4')
ssh_files = fs.glob(bucket)

In [14]:
ssh_files

[]

In [13]:
%%time
fileset = [fs.open(url) for url in ssh_files]
data = xr.open_mfdataset(fileset, combine='by_coords')

OSError: no files to open